# 基礎班時問 1330

問1. 怎麼用python抓share point的資料

程式相關問題通常把問題翻譯成英文的簡短句子丟到 google 裡, 第一筆 stack overflow 的資料有 5, 6 成是解答

https://stackoverflow.com/questions/53671547/python-download-files-from-sharepoint-site

# 影像班時問 1430

In [ ]:
問1.  怎麼使用 GPU 加速 YOLO


- 1. 查 torch 版本
- 2. 安裝相對應的 CUDA toolkit
- 3. 重新安裝 torch 與 cuda 相關的套件

https://developer.nvidia.com/cuda-toolkit-archive/
https://pytorch.org/get-started/previous-versions/

cuda 12.1
pytorch 2.2.2
會使用以下設定
conda install pytorch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 pytorch-cuda=12.1 -c pytorch -c nvidia


In [3]:
import torch

In [22]:
# 確認 cuda 可以用
torch.cuda.is_available()

True

In [2]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8n.pt')

In [21]:
model = model.to('cuda') # 這裡可以指定 使用 cuda

## yolo 模型使用

In [23]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model = model.to('cuda') # 這裡可以指定 使用 cuda


## yolo 模型訓練步驟
- 準備資料
- 選擇模型
- 訓練
- 使用模型


# 綜合時間 1530